# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,62.01,33,40,4.61,US,1715804851
1,1,grytviken,-54.2811,-36.5092,27.01,84,100,15.30,GS,1715804851
2,2,avarua,-21.2078,-159.7750,75.25,88,75,16.11,CK,1715804851
3,3,inhambane,-23.8650,35.3833,73.29,78,89,7.96,MZ,1715804851
4,4,bhinmal,25.0000,72.2500,87.39,41,4,9.48,IN,1715804851


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
# Configure the map plot 
map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    size='Humidity', 
    geo=True, 
    tiles='CartoLight', 
    title='City Humidity Map'
)

# Set options for the plot
map_plot.opts(width=800, height=500)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Define your criteria
temperature_min = 60
temperature_max = 90
wind_speed_max = 15
cloudiness_max = 40

# Filter the DataFrame based on criteria
filtered_df = city_data_df[
    (city_data_df['Max Temp'] > temperature_min) & 
    (city_data_df['Max Temp'] < temperature_max) & 
    (city_data_df['Wind Speed'] < wind_speed_max) &
    (city_data_df['Cloudiness'] == cloudiness_max)
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,62.01,33,40,4.61,US,1715804851
87,87,georgetown,5.4112,100.3354,82.35,85,40,0.00,MY,1715804857
97,97,tamanrasset,22.7850,5.5228,87.71,9,40,6.91,DZ,1715804858
127,127,kalispell,48.1958,-114.3129,69.91,42,40,13.80,US,1715804861
172,172,ribeira grande,38.5167,-28.7000,64.80,68,40,14.97,PT,1715804864


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,33,
87,georgetown,MY,5.4112,100.3354,85,
97,tamanrasset,DZ,22.7850,5.5228,9,
127,kalispell,US,48.1958,-114.3129,42,
172,ribeira grande,PT,38.5167,-28.7000,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key,  
    "format": "json",
    "categories": "accommodation.hotel"
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{lng},{lat}"
    params["bias"] = f"point:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
badger - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
kalispell - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
kill devil hills - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
mao - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
hilo - nearest hotel: No hotel found
callao - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
yorkton - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
san juan de las huertas - nearest hotel: No hotel found
menomonie - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,badger,US,64.8000,-147.5333,33,No hotel found
87,georgetown,MY,5.4112,100.3354,85,No hotel found
97,tamanrasset,DZ,22.7850,5.5228,9,No hotel found
127,kalispell,US,48.1958,-114.3129,42,No hotel found
172,ribeira grande,PT,38.5167,-28.7000,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size='Humidity',
    tiles='CartoLight',
    title='City Humidity Map with Hotel Info',
    hover_cols=['City', 'Country', 'Hotel Name'],
    width=800,
    height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)